In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import os
import pandas as pd

In [5]:
from ordinaloss.utils.datasets import ImageFolder, create_transform_pipeline
from ordinaloss.utils.pretrained_models import classification_model_densenet, classification_model_resnet, classification_model_vgg
from ordinaloss.utils.loss_utils import CSCELoss, create_ordinal_cost_matrix
from ordinaloss.engine.model_engine import OrdinalEngine


import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from torch.optim import Adam, SGD
import torch

/home/dsi/itaym/anaconda3/envs/maruloss/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loaded ordinaloss.utils.pretrained_models


In [6]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [7]:
train_transform = create_transform_pipeline(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.3) #These are hyperparameters. 
test_transform = create_transform_pipeline() #Without color jitter for test pipeline

train_dataset = ImageFolder("../datasets/kneeKL224/train/", transform=train_transform)
val_dataset = ImageFolder("../datasets/kneeKL224/val/", transform = test_transform)
test_dataset = ImageFolder("../datasets/kneeKL224/test/", transform = test_transform)
auto_test_dataset = ImageFolder("../datasets/kneeKL224/auto_test/", transform = test_transform)

train_loader = DataLoader(train_dataset, batch_size = 32, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size = 128, shuffle=False)

In [8]:
#model = classification_model_vgg("vgg16", num_classes = 5) #Could also be resnet34, resnet50, resnet101, resnet152, 
model = classification_model_vgg("vgg16", num_classes = 5) #Could also be resnet34, resnet50, resnet101, resnet152, 
# model = classification_model_densenet("densenet121", num_classes = 5) #Could also be densenet169, densenet201

print("asd!")

In [10]:
def my_lambda_scheduler(epoch):
    return (0.8 ** (epoch // 5))

engine = OrdinalEngine(model, 
                       loss_fn = CSCELoss(create_ordinal_cost_matrix(5)), #CSCELoss model, matan, you know the dril.
                       device = device, #change to gpu, let me know if it doesnt work on GPU.
                       use_lr_scheduler=True, scheduler_lambda_fn = my_lambda_scheduler,
                       optimizer_fn=SGD, lr = 5.0e-4, weight_decay=5.0e-4)

In [11]:
engine.train(train_loader, test_loader, n_epochs = 15)

100%|██████████| 13/13 [00:39<00:00,  3.05s/it, accuracy=0.457, batch_size=1656, loss=26.5, mae=1.07]


{'loss': 26.519251772746948, 'accuracy': 0.4565217399942702, 'mae': 1.0724637715712837}


100%|██████████| 13/13 [00:43<00:00,  3.32s/it, accuracy=0.498, batch_size=1656, loss=28.7, mae=1.14]


{'loss': 28.65351328872828, 'accuracy': 0.49818840579710144, 'mae': 1.1388888888888888}


100%|██████████| 13/13 [01:05<00:00,  5.02s/it, accuracy=0.611, batch_size=1656, loss=20.6, mae=0.97]


...

{'loss': 20.26565989434431, 'accuracy': 0.6950483143617565, 'mae': 0.8079710144927537}


100%|██████████| 13/13 [00:11<00:00,  1.12it/s, accuracy=0.675, batch_size=1656, loss=22.6, mae=0.866]


{'loss': 22.5582665729062, 'accuracy': 0.6751207743865856, 'mae': 0.8659420301372879}


100%|██████████| 13/13 [00:11<00:00,  1.14it/s, accuracy=0.652, batch_size=1656, loss=20.1, mae=1.03]

{'loss': 20.102727539873353, 'accuracy': 0.6515700511886302, 'mae': 1.0307971037528365}
